# Udacity Data Enginering nanodegree capstone poject
### U.S city average temperature and Immigration data Data Lake

#### Project Summary
Generally the scope of this project is to model the data lake allowing users to capture and visualise the changeof ground tempratures throught out the years, now i have chosen to includetemperatures measured even before commercial airtravel was introduced.

We will attempt read, write and analyise data for more than 200k+ travelers
using spark for our U.S immigration data provided by the US National Tourism
and Trade Office, residing on a github repository as well as the World Temperature Data found on kaggle, U.S. City Demographic Data from OpenSoft and finally, U.S Airport Code Table data.

links for the data sources all provided in the last cell.

Deploying a star schema database desing as the immigration data is the fact table vs
the national temprature readings table, the national land air and sea ports table, 
the us city demographics table as dimension tables, and finally a time table extracted from
our immigration data

The last table here is essential, for an example if the city x, maitains a yearly population growth rate of 2%, an average of 1000 tourists a year and an average of +0.0005 degrees a year

On the random year z, we see an increase of 1.0 degrees on april, the data from our demographics table indicates that on the same year we see an increase of 15% in the population
yet we maintain the 1000 tourists a year indicated from out fact table, and in this case we can
conclude that immigration was not the cause of the increase in the temprature recorded.


Or simply to see if we can notice any significant changes in demographics based on immigration.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# imports and installs
import pandas as pd
pd.options.display.max_columns = None

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as F

from pyspark.sql.functions import (year, month, dayofmonth, hour,
weekofyear, date_format, dayofweek)


### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

### Step 2: Explore and Assess the Data
#### Explore the Data 

while we might need to run a scheduled ELT pipeline in most cases of DataLakes,
using airflow to run the extract load and transform processes for this project is so far an option but might get complicated with jupyter and the project's viewer python version among other things to consider.

We have a sample to our immigration data and the full data for the dimensions tables loaded below to determine the data wrangling steps we need to execute.

#### Cleaning Steps

while data analyists prefer well structered and defined data but data scientists prefer raw data or as unchanged as possiple, I will maintain minumum changes to the data set yet allowing for both by definning a proper data type format, the absence of null values and duplicates.

In [2]:
# Read in the data here
demo_df = pd.read_csv('us-cities-demographics.csv', sep =';')
df_ports = pd.read_csv('airport-codes_csv.csv')
migration = pd.read_csv('immigration_data_sample.csv')

fname = '../../data2/GlobalLandTemperaturesByCity.csv'
temp_df = pd.read_csv(fname)

In [3]:
#data sample
demo_df.sample(1)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
2556,Boise,Idaho,34.9,110099.0,108181.0,218280,16004.0,13409.0,2.61,ID,Asian,9806


In [4]:
#dataframe's information
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


In [5]:
#counting any null values
demo_df.isnull().sum()

City                       0
State                      0
Median Age                 0
Male Population            3
Female Population          3
Total Population           0
Number of Veterans        13
Foreign-born              13
Average Household Size    16
State Code                 0
Race                       0
Count                      0
dtype: int64

In [6]:
demo_df.duplicated().sum()

0

In [7]:
#data sample
demo_df.sample(2)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
2500,Riverview,Florida,32.2,44210.0,45536.0,89746,6943.0,12240.0,3.22,FL,Black or African-American,15159
1614,Gastonia,North Carolina,36.9,35527.0,39023.0,74550,3537.0,5715.0,2.67,NC,White,46362


In [8]:
#dataframe's information
df_ports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.0+ MB


In [9]:
#counting any null values
df_ports.duplicated().sum()

0

In [10]:
#data sample
df_ports.sample(1)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
4393,3MD2,heliport,Garrett County Memorial Hospital Heliport,2457.0,NaN,US,US-MD,Oakland,3MD2,NaN,3MD2,"-79.40119934082031, 39.41320037841797"


In [11]:
#data sample
temp_df.sample(1)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
3797009,1856-07-01,21.632,1.302,Khoy,Iran,37.78N,44.75E


In [12]:
#counting any null values
temp_df.duplicated().sum()

0

In [13]:
#data sample
temp_df.sample(1)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
4352253,1797-08-01,30.145,2.104,Loni,India,28.13N,77.27E


In [14]:
#dataframe's information
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
dt                               object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: float64(2), object(5)
memory usage: 459.2+ MB


In [15]:
#data sample
migration.sample(2)

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
271,2163396,4348794.0,2016.0,4.0,525.0,525.0,X96,20567.0,2.0,VQ,20665.0,48.0,2.0,1.0,20160423,SAJ,NaN,G,I,NaN,M,1968.0,10222016,F,NaN,VES,9.435467e+10,MANAD,B2
737,147477,275285.0,2016.0,4.0,209.0,209.0,AGA,20546.0,1.0,GU,20548.0,38.0,2.0,1.0,20160402,NaN,NaN,G,O,NaN,M,1978.0,06302016,M,NaN,DL,5.546549e+10,00608,WT


## Step 3: Defining the Data Model
### 3.1 Conceptual Data Model

#### Spending only a few minutes looking at the dataframes
#### we have above it is pretty much a give on where
#### we weill be getting our data for the fact table vs our dimension tables.

#### again we will be using a star schema fact vs dimension tables design

### With 5 tables in mind : 
#### 1) a fact table - tempratures >> From the spark_migration dataframe<< note that
#### 2) a dimension table - demographics >> From the spark_migration dataframe
#### 3) a dimension table - ports >> From the spark_migration dataframe
#### 4) a dimension table - migrants >> From the spark_migration dataframe 



### 3.2 Mapping Out Data Pipelines

#### Using spark we will perform the tasks above, a few extra steps of wrangling may be included
#### Replacing the country codes by their names and possiply the travel modes while keeping the 
#### original data, which would allow for easier aggregation, grouping and possibly calssifications.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [16]:
#now lets begin our spark session,
spark= SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

#locate and read our immigration data files,
#loading the data into a spark dataframe
spark_migration= spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [17]:
#lets load the previously wrangled data files
spark_port= spark.read.option("header",True).csv('airport-codes_csv.csv')
spark_temp= spark.read.option("header",True).csv(fname)
spark_demo= spark.read.option("header",True).csv('us-cities-demographics.csv', sep=';')

#### Now that we have our main data loaded in spark, lets go ahead load the rest of the data from the files we created, perform the pervious wrangling steps and proceed to creating our data lake.

In [18]:
#dropping unwanted data
spark_migration= spark_migration.drop('Unnamed: 0','occup',
                                       'entdepu','entdepu','insnum','biryear',
                                       'i94yr','i94mon','count')
#dropping al null values
spark_migration= spark_migration.na.drop()

#renaming the columns for an easier query and presentaion
spark_migration= spark_migration.withColumnRenamed('i94cit','from_country')
spark_migration= spark_migration.withColumnRenamed('i94res','from_residence')
spark_migration= spark_migration.withColumnRenamed('i94visa','visa_type')
spark_migration= spark_migration.withColumnRenamed('i94port','port_code')
spark_migration= spark_migration.withColumnRenamed('visapost','port_city')
spark_migration= spark_migration.withColumnRenamed('i94mode','travel_mode')
spark_migration= spark_migration.withColumnRenamed('i94addr','state')
spark_migration= spark_migration.withColumnRenamed('i94bir','age')
spark_migration= spark_migration.withColumnRenamed('dtadfile','date_added')
spark_migration= spark_migration.withColumnRenamed('entdepa','arrival_flag')
spark_migration= spark_migration.withColumnRenamed('entdepd','departure_flag')
spark_migration= spark_migration.withColumnRenamed('matflag','match_flag')
spark_migration= spark_migration.withColumnRenamed('dtaddto','expiry_date')
spark_migration= spark_migration.withColumnRenamed('admnum','admission_number')
spark_migration= spark_migration.withColumnRenamed('fltno','flight_number')


#dropping unwanted columns in the airports data
spark_port = spark_port.drop('continent','iata_code','local_code','elevation_ft',
                      'gps_code','municipality')

#filtering data for the united states only
spark_port = spark_port.filter(spark_port['iso_country']== 'US')

#dropping all null values
spark_port = spark_port.na.drop()

#load the data into a pandas dataframe
spark_port= spark_port.toPandas()

#extract only thr state code from the region column
spark_port['iso_region']= spark_port['iso_region'].str.replace(r'US-','')

spark_port = spark.createDataFrame(spark_port)


#dropping all null value in our demographics data
spark_demo = spark_demo.na.drop()

#renaming the columns for an easier query and presentaion
spark_demo= spark_demo.withColumnRenamed('Male Population','male_population')
spark_demo= spark_demo.withColumnRenamed('Female Population','female_population')
spark_demo= spark_demo.withColumnRenamed('Total Population','total_population')
spark_demo= spark_demo.withColumnRenamed('Number of Veterans','number_of_vets')
spark_demo= spark_demo.withColumnRenamed('Foreign-born','foreign_born')
spark_demo= spark_demo.withColumnRenamed('Average Household Size','avrg_household_size')
spark_demo= spark_demo.withColumnRenamed('Race','race')
spark_demo= spark_demo.withColumnRenamed('State Code','state_code')
spark_demo= spark_demo.withColumnRenamed('Count','count_no')



#filtering temprature data for the united states only
spark_temp = spark_temp.filter(spark_temp['country']== 'United States')

#renaming the columns for an easier query and presentaion
spark_temp= spark_temp.withColumnRenamed('AverageTemperature','avg_temp')
spark_temp= spark_temp.withColumnRenamed('AverageTemperatureUncertainty','avg_temp_uncertainty')


#dropping all null value in our temprature data
spark_temp = spark_temp.na.drop()

In [19]:
#now we fit our data in a temp view to query and creae our finals tables
spark_migration.createOrReplaceTempView('migrations')
spark_temp.createOrReplaceTempView('temps')
spark_demo.createOrReplaceTempView('demos')
spark_port.createOrReplaceTempView('ports')

In [20]:
# queries to select our wanted data tables from the loaded datarames above
# all of the the dates in arrdat are the total days since the first reading on jan 1st,1960
# first commercial trip year
immigration = spark.sql("""
                        SELECT DISTINCT
                          cicid,
                          date_add(to_date('1960-01-01'), arrdate) AS date,
                          state,
                          admission_number,
                          age,
                          gender,
                          visatype AS visa_type,
                          flight_number
                        FROM migrations
                        """
                           )

#fits the dataframe to a tempview
immigration.createOrReplaceTempView("immigrations")


# queries to select our wanted data tables from the loaded datarames above
airports = spark.sql("""SELECT DISTINCT
                         ident,
                         type,
                         name,
                         iso_country,
                         iso_region AS state,
                         coordinates
                         FROM ports 
                        """
                    )

#fits the dataframe to a tempview
airports.createOrReplaceTempView("airports_table")

# queries to select our wanted data tables from the loaded datarames above
demographics = spark.sql("""SELECT DISTINCT
                             count_no,
                             City,
                             State,
                             state_code,
                             male_population,
                             female_population,
                             total_population,
                             number_of_vets,
                             avrg_household_size,
                             race                            
                            FROM demos 
                            """
                        )

#fits the dataframe to a tempview
demographics.createOrReplaceTempView("demographics")

# extrat the city to state data from our demogrphics
city_state = spark.sql("""SELECT DISTINCT
                           city,
                           state_code AS state
                           FROM demographics
                          """
                      )
#fits the dataframe to a tempview
city_state.createOrReplaceTempView("city_state")


# extracting, day, month and year from the dt column for our temprature data
temp_time = spark_temp.select('dt').withColumn('month', month(spark_temp['dt']))\
                                    .withColumn('year', year(spark_temp['dt']))

#drop duplicates if they were created here
temp_time = temp_time.dropDuplicates(subset=['dt'])

#fits the dataframe to a tempview
temp_time.createOrReplaceTempView("temp_time")

#drop null values if any
temp_time = temp_time.na.drop()


#fits the dataframe to a tempview
temp_time.createOrReplaceTempView("temp_time")

# queries to select our wanted data tables from the loaded datarames above
# we will incliude a 15 year gap before airtravel going mainstream for comparisons.
temperatures1 = spark.sql("""SELECT DISTINCT
                                temp_time.dt,
                                temp_time.month,
                                temp_time.year,
                                temps.City,
                                temps.avg_temp,
                                temps.Latitude,
                                temps.Longitude                              
                            FROM temps 
                            JOIN temp_time
                            ON temps.dt = temp_time.dt
                            WHERE year > 1945
                            """
                        )

#fits the dataframe to a tempview
temperatures1.createOrReplaceTempView("temperatures_table")

#drop null values if any
temperatures1= temperatures1.na.drop()


#Add the state code here to fascilitate easier joins and aggregations
# for users
temperatures = spark.sql("""SELECT DISTINCT
                                temperatures_table.month,
                                temperatures_table.year,
                                temperatures_table.City,
                                city_state.state,
                                temperatures_table.avg_temp,
                                temperatures_table.Latitude,
                                temperatures_table.Longitude                              
                            FROM temperatures_table
                            JOIN city_state
                            ON temperatures_table.City = city_state.City
                            """
                        )
temperatures.createOrReplaceTempView("temperatures")
#double joins is possible but wii reduce query speed

### 4.2 Data Quality Checks
#### We will try to run some the queries that might be used by the user.

### The immigration table

In [21]:
#test put a random yet a common type of query on this data.
spark.sql("""
    SELECT
        *
    FROM immigrations
    WHERE state = 'VA' AND age > 45
    """
).show(5)

+--------+----------+-----+----------------+----+------+---------+-------------+
|   cicid|      date|state|admission_number| age|gender|visa_type|flight_number|
+--------+----------+-----+----------------+----+------+---------+-------------+
|274085.0|2016-04-02|   VA|  9.255303493E10|46.0|     M|       B1|        00002|
|432764.0|2016-04-02|   VA|  9.256290193E10|63.0|     F|       B2|        00100|
|502407.0|2016-04-03|   VA|  9.267630183E10|59.0|     F|       B2|        00087|
|716107.0|2016-04-04|   VA|  9.275570343E10|59.0|     F|       B2|        00956|
|835065.0|2016-04-05|   VA|  9.283845283E10|61.0|     M|       B2|        00007|
+--------+----------+-----+----------------+----+------+---------+-------------+
only showing top 5 rows



In [22]:
#view the final schema
immigration.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- date: date (nullable = true)
 |-- state: string (nullable = true)
 |-- admission_number: double (nullable = true)
 |-- age: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- visa_type: string (nullable = true)
 |-- flight_number: string (nullable = true)



In [23]:
#view the first row
immigration.show(n=1, vertical= True)

-RECORD 0--------------------------
 cicid            | 2156.0         
 date             | 2016-04-01     
 state            | CA             
 admission_number | 9.250802923E10 
 age              | 36.0           
 gender           | M              
 visa_type        | B2             
 flight_number    | 00077          
only showing top 1 row



### The main fact table

In [24]:
#test a random yet a common type of query on this data.
spark.sql("""
            SELECT 
                *
            FROM temperatures
            WHERE state = 'VA' AND City = 'Arlington' AND avg_temp >26
            ORDER BY year DESC
            """
).show(5)

+-----+----+---------+-----+--------+--------+---------+
|month|year|     City|state|avg_temp|Latitude|Longitude|
+-----+----+---------+-----+--------+--------+---------+
|    8|2013|Arlington|   VA|  30.255|  32.95N|   96.70W|
|    6|2013|Arlington|   VA|  27.691|  32.95N|   96.70W|
|    7|2013|Arlington|   VA|   28.49|  32.95N|   96.70W|
|    9|2013|Arlington|   VA|  27.986|  32.95N|   96.70W|
|    6|2012|Arlington|   VA|  28.142|  32.95N|   96.70W|
+-----+----+---------+-----+--------+--------+---------+
only showing top 5 rows



In [25]:
#view the final schema
temperatures.printSchema()

root
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- state: string (nullable = true)
 |-- avg_temp: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [26]:
#view the first row
temperatures.show(n=1, vertical= True)

-RECORD 0------------
 month     | 3       
 year      | 1955    
 City      | Detroit 
 state     | MI      
 avg_temp  | 1.236   
 Latitude  | 42.59N  
 Longitude | 82.91W  
only showing top 1 row



#### The airports table

In [27]:
#test a random yet a common type of query on this data
spark.sql("""
    SELECT
        name,
        state,
        type
    FROM airports_table
    WHERE state = 'NY'
    ORDER BY type DESC
    """
         ).show(5)

+--------------------+-----+-------------+
|                name|state|         type|
+--------------------+-----+-------------+
|      Merkle Airport|   NY|small_airport|
|UVMHN-Elizabethto...|   NY|     heliport|
|Middlesex Valley ...|   NY|small_airport|
|     Skyview Airport|   NY|small_airport|
|H & H Aviation Se...|   NY|     heliport|
+--------------------+-----+-------------+
only showing top 5 rows



In [28]:
#view the final schema
airports.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [29]:
#view the first row
airports.show(n=1, vertical= True)

-RECORD 0---------------------------
 ident       | 0AL7                 
 type        | heliport             
 name        | Gadsden Regl Medi... 
 iso_country | US                   
 state       | AL                   
 coordinates | -85.9653015136718... 
only showing top 1 row



### the demographics table.

In [50]:
#view the final schema
demographics.printSchema()

root
 |-- count_no: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- male_population: string (nullable = true)
 |-- female_population: string (nullable = true)
 |-- total_population: string (nullable = true)
 |-- number_of_vets: string (nullable = true)
 |-- avrg_household_size: string (nullable = true)
 |-- race: string (nullable = true)



In [51]:
#view the first row
demographics.show(n=1, vertical= True)

-RECORD 0-----------------------------------
 count_no            | 124108               
 City                | Greensboro           
 State               | North Carolina       
 state_code          | NC                   
 male_population     | 132251               
 female_population   | 153093               
 total_population    | 285344               
 number_of_vets      | 14011                
 avrg_household_size | 2.36                 
 race                | Black or African-... 
only showing top 1 row



In [49]:
#test a random yet a common type of query on this data
spark.sql("""
    SELECT
        *
    FROM demographics
    WHERE City IS NULL
    """
         ).show(vertical=True)

(0 rows)



#### As expected we got no result of rentries where is the city is null
#### also here i kept most of the as string to keep each entry's uniqness
#### casting as int or floats and, ML users can now both use this data with min-wrangle.

In [52]:
# saving the new data lake locally as .sas files
immigration.write.parquet('data_lake/us_immigration.sas')
temperatures.write.parquet('data_lake/us_city_temperatures.sas')
airports.write.parquet('data_lake/us_airports.sas')
demographics.write.parquet('data_lake/us_city_demographics.sas')

### Step 5: Complete Project Write Up

#### 1) for the clear size of our data, we have mainly deployed Pyspark to shape and process our data model and used pandas to understand the shape and get a feeling to the data in a broader way,

#### 2) In the case of a constant stream of data from each source, Applying similar functions and technology, we can move this to airflow and schedule it of a daily or even an hourly basis depending on the decesions taken supported by the analysis used buy this datalake

#### 3)under the following scenarios:

#### -The data was increased by 100x:

#### instantly move the spark session to an EMR or AWS, using AWS S3 for its cost effectivenes.

#### -The data populates a dashboard that must be updated on a daily basis by 7am every day:

#### In this case we would move the storage back to hadoop and maintain an active EMR cluster, using apache airflow to process the ELT on a daily basis at 7 am.

#### -The database needed to be accessed by 100+ people:

#### With a deployed EMR cluster on AWS, depending on the tools used by the user, using an ssh key and a few other steps dependinng on which software used, like power BI and tablue, which both are suported BY AWS and they include AWS integration prefrences as well.

### Data Dict.

#### The Immigration table
##### cicid >> unique number given to each arrival       
##### date  >> arrival date
##### state >> arrival state        
##### admission_number >> number given to traveller by customs
##### age  >>  traveler's age      
##### gender >> traveler's gender         
##### visa_type >> tyoe of visa given to taveller
##### flight_number >> flight number the traveller was on.       

#### The Temperature table
##### month >> month at the time of calulating the average temperatures   
##### year >> year at the time of calulating the average temperatures
##### City >> city measuerment was taken in 
##### state >> state measuerment was taken in      
##### avg_temp >> average temperature calculated
##### Latitude >> gps and map location of the data capture
##### ngitude >> gps and map location of the data capture

#### The Airports table 
##### ident >> airport id serial number          
##### type  >> the type of port, heliports, frieght,...etc         
##### name >> the airport's name
##### state  >> state the airport is in              
##### coordinates >> map coordinations of the airport

#### The demographics table
##### count_no  >> the number of the count processes since the first time at a given city          
##### City >> city of the  population count       
##### State  >> the state the city is in
##### state_code  >>  the state two letter code like VA, TX, PA ..etc     
##### male_population  >> female population of the city          
##### female_population >>  male population of the city        
##### total_population  >> total population of the city           
##### number_of_vets >> average number of verts in the city             
##### avrg_household_size  >> the average household size in the city          
##### race >> the majority race of the city






I94 Immigration Data:
https://travel.trade.gov/research/reports/i94/historical/2016.html

World Temperature Data:
https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data

U.S. City Demographic Data:
https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/

Airport Code Table:
https://datahub.io/core/airport-codes#data

##### refrences:
https://www.marsja.se/how-to-read-sas-files-in-python-with-pandas/

https://stackoverflow.com/questions/49188960/how-to-show-all-columns-names-on-a-large-pandas-dataframe

https://towardsdatascience.com/data-wrangling-with-pyspark-for-beginners-3f2197c81511

https://stackoverflow.com/questions/43976237/pyspark-replacing-value-in-a-column-by-searching-a-dictionary

https://sparkbyexamples.com/pyspark/pyspark-cast-column-type/

https://stackoverflow.com/questions/39067505/pyspark-display-a-spark-data-frame-in-a-table-format

https://stackoverflow.com/questions/50523950/improve-pyspark-dataframe-show-output-to-fit-jupyter-notebook

https://stackoverflow.com/questions/37038014/pyspark-replace-strings-in-spark-dataframe-column

https://stackoverflow.com/questions/66311283/replacing-column-values-in-pyspark-by-iterating-through-list

https://stackoverflow.com/questions/52598366/pyspark-replace-all-values-in-dataframe-with-another-values

https://sparkbyexamples.com/spark/spark-functions-adding-days-months-year/